In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib
import glob
import os
from datetime import date, datetime, time
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

from scipy.stats import trim_mean
from statsmodels import robust
from matplotlib import font_manager, rc
%matplotlib inline
from matplotlib import cm 
import gc  # garbage collection
from sklearn.cluster import KMeans  # k-means 군집분석
from sklearn.metrics import silhouette_score  # 군집분석 평가를 위한 silhouette 값 계산

pd.set_option('max_columns', 20, 'max_rows', 20)

%matplotlib inline

# 한글 폰트 설정|
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

import datetime
import time

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
cus_info = pd.read_csv('../data/cus_info.csv')
iem_info = pd.read_csv('../data/iem_info.csv')
stk_bnc_hist = pd.read_csv('../data/stk_bnc_hist.csv')
train = pd.read_csv('../data/stk_hld_train.csv')
test = pd.read_csv('../data/stk_hld_test.csv')

In [4]:
stk_bnc_hist

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0
...,...,...,...,...,...,...
2573834,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200806,A035720,1.0,364000.0,500.0
2573835,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200813,A035720,0.0,0.0,500.0
2573836,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200819,A035720,1.0,376500.0,500.0
2573837,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200825,A035720,0.0,0.0,500.0


In [5]:
#stk_bnc_hist = stk_bnc_hist.drop(stk_bnc_hist[stk_bnc_hist.bnc_qty == 0].index)

In [6]:
stk_bnc_hist = stk_bnc_hist.query('tot_aet_amt >= 0')

In [7]:
stk_bnc_hist

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0
...,...,...,...,...,...,...
2573834,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200806,A035720,1.0,364000.0,500.0
2573835,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200813,A035720,0.0,0.0,500.0
2573836,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200819,A035720,1.0,376500.0,500.0
2573837,720aa28d24bfa8fbeddf4fe625cb53af0a6f7ff2d526de...,20200825,A035720,0.0,0.0,500.0


In [8]:
stk_bnc_hist = stk_bnc_hist.rename(columns = {'act_id': '계좌번호',
                                              'bse_dt': '기준일자',
                               'iem_cd': '종목코드',
                               'bnc_qty':'종목보유잔고수량',
                              'tot_aet_amt': '전체종목평가금액',
                              'stk_par_pr': '주당액면가'})

In [9]:
stk_bnc_hist = stk_bnc_hist.sort_values(by=['계좌번호','종목코드', '기준일자'])

- 계좌별_종목별_일_매수매도_수량_평균 -> 한번에 얼마나 많은 수량을 거래하느냐를 알고자 함  
- 계좌별_종목별_매수매도_금액_차액 -> 계좌가 한 종목으로 얼마나 많은 수익, 손해를 봤는지를 알고자 함 --> 이건 불가능한가...?
- 계좌별_종목별_평균_보유기간 -> 기준일차 첫번째에서 마지막을 뺀 기간을 확인
- 계좌별_종목별_첫번째매수_마지막매도_차액 -> 어느정도의 차액이 생겨야 매수 매도 함을 알고자 함

In [10]:
final_df = pd.DataFrame(columns = ['계좌번호','기준일자','종목코드','주당액면가','종목보유잔고수량','종목보유잔고수량_shift','전체종목평가금액','전체종목평가금액_shift','일_매수매도_수량','일_매수매도_총금액'])
cnt = 0
for i in tqdm(list(stk_bnc_hist.계좌번호.unique()), desc="진행사항", mininterval=0.01):
    sam_df = stk_bnc_hist[stk_bnc_hist.계좌번호 == i]
    for j in list(sam_df.종목코드.unique()):
        sam_sam_df = sam_df[sam_df.종목코드 == j]
        sam_sam_df['종목보유잔고수량_shift'] = sam_sam_df.종목보유잔고수량.shift(1).fillna(0)
        sam_sam_df['전체종목평가금액_shift'] = sam_sam_df.전체종목평가금액.shift(1).fillna(0)
        sam_sam_df['일_매수매도_수량'] = sam_sam_df['종목보유잔고수량'] - sam_sam_df['종목보유잔고수량_shift']
        sam_sam_df['일_매수매도_총금액'] = sam_sam_df['전체종목평가금액'] - sam_sam_df['전체종목평가금액_shift']
        sam_sam_df['일_매수매도_주당금액'] = sam_sam_df['일_매수매도_총금액'] / sam_sam_df['일_매수매도_수량']
        sam_sam_df = sam_sam_df.reset_index(drop=True)
        sam_sam_df['계좌별_종목별_일_매수_수량_평균'] = sam_sam_df[sam_sam_df.일_매수매도_수량 > 0].일_매수매도_수량.mean()
        sam_sam_df['계좌별_종목별_일_매도_수량_평균'] = -sam_sam_df[sam_sam_df.일_매수매도_수량 < 0].일_매수매도_수량.mean()
        b = sam_sam_df['기준일자'].iloc[0]
        c = sam_sam_df['기준일자'].iloc[-1]
        sam_sam_df['계좌별_종목별_총보유기간'] = (datetime.datetime.strptime(str(c), '%Y%m%d') - datetime.datetime.strptime(str(b), '%Y%m%d')).days
        sam_sam_df['계좌별_종목별_마지막처음_차액'] = (sam_sam_df.전체종목평가금액_shift / sam_sam_df.종목보유잔고수량_shift).iloc[-1] - (sam_sam_df.전체종목평가금액 / sam_sam_df.종목보유잔고수량).iloc[0]        
        final_df = pd.concat([final_df, sam_sam_df])

진행사항: 100%|██████████| 9992/9992 [25:17:36<00:00,  9.11s/it]   


In [11]:
final_df

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,일_매수매도_총금액,일_매수매도_주당금액,계좌별_종목별_일_매수_수량_평균,계좌별_종목별_일_매도_수량_평균,계좌별_종목별_총보유기간,계좌별_종목별_마지막처음_차액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,20190321,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,667000.0,667000.0,1.666667,5.0,516.0,-313500.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,20190328,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,603000.0,603000.0,1.666667,5.0,516.0,-313500.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,20190507,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,545000.0,545000.0,1.666667,5.0,516.0,-313500.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,20190716,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,289000.0,289000.0,1.666667,5.0,516.0,-313500.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,20200129,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,860000.0,215000.0,1.666667,5.0,516.0,-313500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,20201013,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,-2300.0,-2300.0,1.250000,5.0,91.0,-24700.0
3,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,20201124,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,48900.0,48900.0,1.250000,5.0,91.0,-24700.0
4,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,20201214,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,-245500.0,49100.0,1.250000,5.0,91.0,-24700.0
0,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,20200910,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,1072500.0,71500.0,15.000000,15.0,5.0,0.0


In [24]:
final_df.to_csv('../data/0930피쳐.csv', encoding='cp949', index=False)